# 不使用预训练词向量模型的情感分类

In [1]:
import numpy as np

In [2]:
# 定义文档集
documents = ['Well done!',
             'Good work',
             'Great effort',
             'nice work',
             'Excellent!',
             'Weak',
             'Poor effort!',
             'not good',
             'poor work',
             'Could have done better.']

# 定义类别标记
labels = np.array([1,1,1,1,1,0,0,0,0,0])

## One-Hot编码词向量

In [3]:
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


这是一个对hashing_trick函数的包装器，该函数使用 hash将词语映射为一个索引，但不保证映射单一性。

In [4]:
vocab_size = 50
encodeDocuments = [one_hot(doc, vocab_size) for doc in documents]
print(encodeDocuments)

[[15, 30], [28, 1], [4, 2], [39, 1], [37], [22], [28, 2], [10, 28], [28, 1], [5, 42, 30, 40]]


显然，一个句子中有几个单词，经过one_hot方法后就会得到几个索引数值。

## 固定句子长度

In [5]:
from keras.preprocessing.sequence import pad_sequences

In [6]:
max_length = 4
# 从后边填充截断
paddedDocuments = pad_sequences(encodeDocuments, maxlen=max_length, padding='post')
print(paddedDocuments)

[[15 30  0  0]
 [28  1  0  0]
 [ 4  2  0  0]
 [39  1  0  0]
 [37  0  0  0]
 [22  0  0  0]
 [28  2  0  0]
 [10 28  0  0]
 [28  1  0  0]
 [ 5 42 30 40]]


## 构造网络模型并编译

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [8]:
model = Sequential()
# 添加嵌入层
model.add(Embedding(vocab_size,                # 词汇表大小决定嵌入层参数矩阵的行数
                    8,                         # 输出每个词语的维度为8 
                    input_length=max_length))  # 输入矩阵一个句子向量含有的词语数即列数
# 扁平化
model.add(Flatten())
# 输出层
model.add(Dense(1, activation='sigmoid'))
# 编译模型
model.compile(optimizer='adam',              # 优化方法
              loss='binary_crossentropy',    # 损失函数
              metrics=['acc'])               # 效果评估方法
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


## 训练

In [9]:
# 注意epochs数量增加则精度会提高，但是会有过拟合风险
model.fit(paddedDocuments, labels, epochs=50, verbose=0)

## 评估

In [10]:
loss, accuracy = model.evaluate(paddedDocuments, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998
